In [1]:
import pandas as pd
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import sys
sys.path.append("..")
from util import print_answer

## 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv

In [2]:
train = pd.read_csv("salary-train.csv")
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


## 2. Проведите предобработку:

Приведите тексты к нижнему регистру. Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.

In [3]:
def text_transform(text: pd.Series) -> pd.Series:
    return text.str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [4]:
vec = TfidfVectorizer(min_df=5)
X_train_text = vec.fit_transform(text_transform(train["FullDescription"]))

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [5]:
train["LocationNormalized"].fillna("nan", inplace=True)
train["ContractTime"].fillna("nan", inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [6]:
enc = DictVectorizer()
X_train_cat = enc.fit_transform(train[["LocationNormalized", "ContractTime"]].to_dict("records"))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [7]:
X_train = hstack([X_train_text, X_train_cat])

## 3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241.

Целевая переменная записана в столбце SalaryNormalized.

In [8]:
y_train = train["SalaryNormalized"]
model = Ridge(alpha=1, random_state=241)
model.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

## 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [9]:
test = pd.read_csv("salary-test-mini.csv")

X_test_text = vec.transform(text_transform(test["FullDescription"]))
X_test_cat = enc.transform(test[["LocationNormalized", "ContractTime"]].to_dict("records"))
X_test = hstack([X_test_text, X_test_cat])

y_test = model.predict(X_test)
print_answer(1, f"{y_test[0]:.2f} {y_test[1]:.2f}")

56555.62 37188.32
